## Exploratory data analysis
### In this section

- downlad sesmic data and models
- horisonatal projections of wells and seismic cube 
- visualize seismic slices 
- process las files adding well coordinates 
- generate training data - project wells on vertical seismic slices 
- visualize training data: seismic slices, carotage masks (used in loss funstion), target carotage data 

### Download the dataset

In [ ]:
! sh ../src/download_data.sh

In [ ]:
# reload modules automatically:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings(action='ignore')

# make user code available:
import sys
from pathlib import Path
module_path = Path('.').absolute().parent
if str(module_path) not in sys.path:
    sys.path.insert(0, str(module_path / 'src'))

import numpy as np
import pandas as pd
import pickle
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from data_types import Point
from utils import projection
from create_dataset import (
    generate_logs, gen_tgt_mask,
    project_wells_onto_slice,
    create_slice_well_list,
    process_single_wells,
    dump_normalization_values,
    slice_crossline, slice_inline,
    preprocess
)
from const import (
    log_dir, # processed well logs
    slices_dir, # path to generated training data
    slice_coord_path, # dictionary with ilines, xlines coordinates
    trace_coords, # coordinates of seismic traces
    ilines, xlines,
    nsamples, dt, # seismic cube parameters
    wells, wellheads, # well names and well heads coordinates
    norm_dict_path, # narmalization dictionary for neural net
    well_width, # width of vertical well prjection, in seismic traces
    carotage_types,
    model_dir, # path to models
)
from predict import (
    predict_on_fold,
    eval_fold,
    process_all_folds,
    average_prediction,
    cv_dataset
)

In [ ]:
# seismic cube parameters
print(f'# samples={nsamples}, dt={dt} msec')
print(f'min iline={min(ilines)}, max iline={max(ilines)}')
print(f'min xline={min(xlines)}, max xline={max(xlines)}')

### Horisonatal projection of wells and seismic traces

In [ ]:
# read ilines and xlines coordinates
with open(slice_coord_path, 'rb') as f:
    slice_coord_dict = pickle.load(f)

In [ ]:
xline = 387
xline_coords = slice_coord_dict['xline'][xline]

fig = plt.figure(figsize=(10, 10))
plt.scatter(trace_coords[::1, 0], trace_coords[::1, 1], marker='.')
plt.scatter(xline_coords[:, 0], xline_coords[:, 1], marker='.', s=20, c='y')
plt.scatter(wellheads['X-Coord'], wellheads['Y-Coord'], marker='o', s=100, c='k')
for id_, (x_, y_) in wellheads.iterrows():
    plt.annotate(id_, xy=[x_, y_], xytext=(10, 3), textcoords='offset points', c='w', fontsize=16)
plt.legend(['Seismic traces', f'xline {xline}', 'Wells'], loc='lower right')
plt.title(f'Horisonatal projection of well heads')
plt.xlabel('X, m', fontsize=12)
plt.ylabel('Y, m', fontsize=12)
plt.show()

#### Well F03-4 projection on inline 730

In [ ]:
well_name = 'F03-4'
slice_type = 'iline'
slice_num = 730

slice_coords = slice_coord_dict[slice_type][slice_num]

ptw = Point(*wellheads.loc[well_name, ['X-Coord', 'Y-Coord']])
pt1 = Point(*slice_coords[0])
pt2 = Point(*slice_coords[-1])
ptp = projection(pt1, pt2, ptw)

fig = plt.figure(figsize=(10, 10))
plt.scatter(trace_coords[::1, 0], trace_coords[::1, 1], marker='.')
plt.scatter(slice_coords[:, 0], slice_coords[:, 1], marker='.', s=20, c='y')
plt.scatter(*pt1, marker='o', s=100, c='r')
plt.scatter(*pt2, marker='o', s=100, c='g')
plt.scatter(*ptw, marker='o', s=100, c='k')
plt.scatter(*ptp, marker='o', s=100, c='c')
plt.legend(['seismic traces', f'{slice_type} {slice_num}', 'start', 'end', f'head of {well_name}', f'{well_name} projection on {slice_type}'], loc='lower right')
plt.title(f'Horisonatal projection of well {well_name} on {slice_type} {slice_num}')
plt.xlabel('X, m', fontsize=12)
plt.ylabel('Y, m', fontsize=12)
plt.show()

### Visualize inline

In [ ]:
iline = 730
seismic_slice = slice_inline(iline)

fig = plt.figure(figsize=(10, 15))
plt.imshow(seismic_slice, cmap = 'seismic', aspect = 'auto')
plt.axis('off')
plt.title(f'Inline {iline}')
plt.show()

### Visualize crossline

In [ ]:
xline = 400
seismic_slice = slice_crossline(xline)

fig = plt.figure(figsize=(10, 15))
plt.imshow(seismic_slice, cmap = 'seismic', aspect = 'auto')
plt.axis('off')
plt.title(f'Crossline {xline}')
plt.show()

### Preprocess and extend log files. Generate training data.

In [ ]:
preprocess()

### Plot carotage data

In [ ]:
well_name = 'F03-4'
las_df = pd.read_csv(log_dir / (well_name + '.csv'))

In [ ]:
fontsize = 14
fig, axes = plt.subplots(1, 3, figsize=(10, 20), sharey=True)
las_df.plot(y='t', x='Sonic', ax=axes[0], legend=None, fontsize=fontsize)
las_df.plot(y='t', x='Gamma_Ray', ax=axes[1], legend=None, fontsize=fontsize)
las_df.plot(y='t', x='Porosity', ax=axes[2], legend=None, fontsize=fontsize)
axes[0].set_ylabel('ms', fontsize=fontsize)
axes[0].grid('on')
axes[1].grid('on')
axes[2].grid('on')
plt.gca().invert_yaxis()

### Project single well onto seismic slice

In [ ]:
well_name = 'F03-4'
iline = 730
iline_coords = slice_coord_dict['iline'][iline]
las_df = pd.read_csv(log_dir / (well_name + '.csv'))
vertical_grid = np.arange(nsamples) * dt
target, mask = gen_tgt_mask(iline_coords, vertical_grid, las_df, carotage_types=['P_Impedance'], well_width=20)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 30), sharey=True)
extent = [min(xlines), max(xlines), nsamples * dt - 1, 0]
ax1.imshow(mask[..., 0], cmap='Greys_r', extent=extent)
ax1.set_title('Mask')
ax1.set_ylabel('ms', fontsize = '10')
ax1.set_xlabel('xline', fontsize = '10')
ax2.imshow(target[..., 0], cmap='Spectral_r', extent=extent)
ax2.set_title('P_Impedance(Kg/m2s)')
ax2.set_xlabel('xline', fontsize = '10')
plt.tight_layout()
plt.show()

### Project multiple wells

In [ ]:
well_list = ['F02-1', 'F03-2']
xline = 740
carotage_list = ['Gamma_Ray', 'P_Impedance']
target, mask = project_wells_onto_slice(xline, 'xline', well_list, carotage_types=carotage_types, well_width=20, verbose=True)

### Vizualise mask and carotage target

In [ ]:
extent = [min(ilines), max(ilines), nsamples * dt - 1, 0]
carotage = 'Gamma_Ray'
index = carotage_list.index(carotage)

fontsize = 14
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 30), sharey=True)
ax1.imshow(mask[..., index], cmap='Greys_r', extent=extent)
ax1.set_title('Mask')
ax1.set_ylabel('ms', fontsize=fontsize)
ax1.set_xlabel('iline', fontsize=fontsize)
ax2.imshow(target[..., index], cmap='Spectral_r', extent=extent)
ax2.set_title(f'{carotage} carotage')
ax2.set_xlabel('iline', fontsize=fontsize)
plt.tight_layout()
plt.show()

In [ ]:
carotage = 'P_Impedance'
index = carotage_list.index(carotage)

fontsize = 14
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 30), sharey=True)
ax1.imshow(mask[..., index], cmap='Greys_r', extent=extent)
ax1.set_title('Mask')
ax1.set_ylabel('ms', fontsize=fontsize)
ax2.set_xlabel('iline', fontsize=fontsize)
ax2.imshow(target[..., index], cmap='Spectral_r', extent=extent)
ax2.set_title(f'{carotage} carotage')
ax2.set_xlabel('iline', fontsize=fontsize)
plt.tight_layout()
plt.show()

### Visualize training data

In [ ]:
fl = 'iline_732_F03-2.pkl'
extent = [min(ilines), max(ilines), nsamples * dt - 1, 0] if 'xline' in fl else [min(xlines), max(xlines), nsamples * dt - 1, 0]
with open(slices_dir / fl, 'rb') as f:
    slice_data = pickle.load(f)

fontsize = 14
carotage_type = 'P_Impedance'
seismic_ver = slice_data['seismic']
mask_ver = slice_data['projections'][carotage_type]['mask']
target_ver = slice_data['projections'][carotage_type]['target']

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 14), sharey=True)
ax1.imshow(seismic_ver, cmap='seismic', extent=extent)
ax1.set_title('Seismogram', fontsize=fontsize)
ax1.set_ylabel('ms', fontsize=fontsize)
ax1.set_xlabel('iline' if 'xline' in fl else 'xline', fontsize=fontsize)

ax2.imshow(mask_ver, cmap='Greys_r', extent=extent)
ax2.set_xlabel('iline' if 'xline' in fl else 'xline', fontsize=fontsize)
ax2.set_title('Mask', fontsize=fontsize)

ax3.imshow(target_ver, cmap='Spectral_r', extent=extent)
ax3.set_xlabel('iline' if 'xline' in fl else 'xline', fontsize=fontsize)
ax3.set_title(f'{carotage_type} carotage', fontsize=fontsize)
plt.tight_layout()
plt.show()

In [ ]:
fl = 'xline_1006_F03-4.pkl'
extent = [min(ilines), max(ilines), nsamples * dt - 1, 0] if 'xline' in fl else [min(xlines), max(xlines), nsamples * dt - 1, 0]
with open(slices_dir / fl, 'rb') as f:
    slice_data = pickle.load(f)

fontsize = 14
carotage_type = 'Gamma_Ray'
seismic_ver = slice_data['seismic']
mask_ver = slice_data['projections'][carotage_type]['mask']
target_ver = slice_data['projections'][carotage_type]['target']

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 14), sharey=True)
ax1.imshow(seismic_ver, cmap='seismic', extent=extent)
ax1.set_title('Seismogram', fontsize=fontsize)
ax1.set_ylabel('ms', fontsize=fontsize)
ax1.set_xlabel('iline' if 'xline' in fl else 'xline', fontsize=fontsize)

ax2.imshow(mask_ver, cmap='Greys_r', extent=extent)
ax2.set_xlabel('iline' if 'xline' in fl else 'xline', fontsize=fontsize)
ax2.set_title('Mask', fontsize=fontsize)

im = ax3.imshow(target_ver, cmap='Spectral_r', extent=extent)
ax3.set_xlabel('iline' if 'xline' in fl else 'xline', fontsize=fontsize)
ax3.set_title(f'{carotage_type}', fontsize=fontsize)

axins = inset_axes(ax3, width='5%',   height='30%', loc='upper left')
fig.colorbar(im, cax=axins)

plt.tight_layout()
plt.show()

## Evaluation

### In this section:  
* cross-validate Gamma Ray carotage models on 4 wells from the Netherlands Offshore F3 Block open project, https://terranubis.com/datainfo/Netherlands-Offshore-F3-Block-Complete  
* Dictionary `weights` provides list of 4 models ordered according to test folds. Every test fold associated with just one well -  F02-1, F03-2, F03-4, F06-1. Each of the wells is projected onto 42 nearest seismic slices (2 inlines/crosslines x 21 slices) 
* average well carotage across multiple slices and calculate statistics  
* visualize predictions: carotage for a single well and carotage for a whole seismic slice   

In [ ]:
# trained models in the order of cv test folds
weights = {
    'Gamma_Ray': [
        'uResNet34.Gamma_Ray.sz480x512.smtd_0.14-0.78.hdf5',
        'uResNet34.Gamma_Ray.sz480x512.smtd_1.11-0.37.hdf5',
        'uResNet34.Gamma_Ray.sz480x512.smtd_2.07-0.65.hdf5',
        'uResNet34.Gamma_Ray.sz480x512.smtd_3.42-0.67.hdf5'
    ],
}

### Evaluate models for Gamma Ray carotage

In [ ]:
carotage = 'Gamma_Ray'
eval_dict = process_all_folds(weights, carotage)

#### Average predicted carotage for a well across all slices associated with this well

In [ ]:
mean_eval_dict = average_prediction(eval_dict)
all_correlations = [v['corr'] for v in mean_eval_dict.values()]
print(f'\nOverall correlation: {np.mean(all_correlations):0.2f}')

### Visualize

#### Prediction distribution across all slices

In [ ]:
correlations = [v['corr'] for v in eval_dict.values()]
fig = plt.figure(figsize=(10, 8))
plt.hist(correlations, bins=20, edgecolor='black', linewidth=1.0)
plt.grid('on')
plt.title('Correlation distribution across all slices')
plt.show()

In [ ]:
fontsize = 12
fig, ax = plt.subplots(1, 4, figsize=(14, 14), sharey=True)

for i, (w, ax_) in enumerate(zip(wells, ax.flatten())):
    t = mean_eval_dict[w]['t']
    true_carotage = mean_eval_dict[w]['true_carotage']
    pred_carotage = mean_eval_dict[w]['pred_carotage']
    corr = mean_eval_dict[w]['corr']
    ax_.plot(true_carotage, t, pred_carotage, t)
    ax_.set_title(f'{w}', fontsize=fontsize)
    if i == 0:
        ax_.set_ylabel('ms', fontsize=fontsize)
        ax_.legend([f'Actual {carotage}', 'Predicted'], fontsize=fontsize)
    ax_.text(0.65, 0.02, f'corr={corr:0.2f}', transform=ax_.transAxes, fontsize=fontsize)
    ax_.grid(True)
ax_.invert_yaxis()    
    
plt.tight_layout()
plt.show()

### Visualize predicted Gamma Ray carotage for a whole seismic slice  
#### well F02-1 is associated with test fold 0 and projected onto 42 nearest seismic slices; one of them - inline 363

In [ ]:
fold = 0
slice_list = cv_dataset[fold]['test']
carotage = 'Gamma_Ray'
data = predict_on_fold(slice_list, carotage, model_dir / weights[carotage][fold], verbose=False)

In [ ]:
i_d = 'iline_363_F02-1'

color_norm = colors.Normalize(vmin=data[i_d]['y_pred'].min(), vmax=data[i_d]['y_pred'].max())
extent = [min(ilines), max(ilines), nsamples * dt - 1, 0] if 'xline' in i_d else [min(xlines), max(xlines), nsamples * dt - 1, 0]

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 8), sharey=True)
ax1.imshow(data[i_d]['seism'], cmap='seismic', extent=extent)
ax1.set_title('Seismic slice')
ax1.set_ylabel('ms', fontsize=fontsize)
ax1.set_xlabel('iline' if 'xline' in i_d else 'xline', fontsize=fontsize)

ax2.imshow(data[i_d]['y_true'], cmap='Greys_r', extent=extent, norm=color_norm)
ax2.set_title(f'Actual {carotage} slice')
ax2.set_xlabel('iline' if 'xline' in i_d else 'xline', fontsize=fontsize)

ax3.imshow(data[i_d]['y_pred'], cmap='Greys_r', extent=extent, norm=color_norm)
ax3.set_title(f'Prediced {carotage} slice')
ax3.set_xlabel('iline' if 'xline' in i_d else 'xline', fontsize=fontsize)

plt.tight_layout()
plt.suptitle(f'{{}} {{}}, well {{}}'.format(*i_d.split('_')), fontsize=14, y=0.99)
plt.show()